## **Diffusion Equation**

$$
\frac{d\Sigma}{dt} = \frac{3}{r}\frac{\partial}{\partial r} \left[r^{1/2} \frac{\partial}{\partial r}(\nu \Sigma r^{1/2})\right]
$$

This is evolution equation for the surface density of a geometrically thin disk

Putting this in the form of a diffusion equation:

$$
\frac{\partial f}{\partial t} = D \frac{\partial^2 f}{\partial X^2}
$$
where $X \equiv 2r^{1/2}$, $f \equiv \frac{3}{2}\Sigma X$ and $D = \frac{12\nu}{X^2}$


We need numerical values for $\nu$
$$
\nu = \alpha c_s H \\
H = \frac{c_s}{\Omega} \\
\Omega = \sqrt{\frac{G*M_s}{r^3}} \\
c_s = \sqrt{\frac{k_B * T}{\mu * mH}} \\
mH = \frac{1}{N_A} \\
\mu = 2.3 \\
$$

The Temperature profile is given by:
$$
2 \sigma_b T_{disk}^4 = \frac{9}{4}\Sigma \nu \Omega^2\\
T^3 = \frac{9}{8\sigma_b}\Sigma \frac{k_b \alpha}{\mu mH}\sqrt{\frac{GM}{r^3}}
$$

The Pressure is given by:

$$
P = \frac{c_s^2\Sigma}{H} = c_s\Sigma\Omega
$$


$$St_{max} = \frac{Q*}{\alpha c_s^2}$$


## **Pebble Accretion**

Mass Accretion is given by:
$\dot{M}_{3D} = \pi R_{acc}\rho_{p}\delta v$

Accretion Criterion:
$\tau_f = \cfrac{\xi_B \Delta v + \xi_H \Omega \hat{R}_{acc}}{GM/\hat{R}^2_{acc}}$

$\hat{R}^2_{acc} = \left(\frac{4\tau_f}{t_B}\right)^{1/2} R_B$

$R_B = \frac{GM}{\Delta v^2}$

$t_B = \frac{R_B}{\Delta v}$

$\tau_f = \frac{St}{\Omega}$

$\xi_B = \xi_H = 0.25$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba
import pandas as pd
import os
import sys
from dust_diffuse import *
from thermal import *
from pebble import *
import matplotlib.cm as cm
import matplotlib.patches as ptch
from matplotlib import rc
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import pylab
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

In [2]:
#Natural Constants
# T = 600 # Temperature
mu = 2.2 # mean molar mass in disk
avog = 6.02214 * 10**23 # avogadros number
mH = 1.673534 * 10**-27 # atomic hydrogen mass
kB = 1.380649 * 10**-23 #boltzmann constant
G = 6.6738 * 10**-11 #gravitational constant
Ms = 1.9886 * 10**30 #solar mass
Me = 5.972 * 10 **24 #earth mass
AU = 1.4959787 * 10**11 #astronomical units
yrs2sec = 3.1536 * 10**7 #convert years to seconds
sb = 5.6704*10**-8

In [3]:
#Disk Parameters
alpha1 = 1e-2
alpha2 = 1.0 * 10**-3
alpha3 = 1.0 * 10**-4
rin = 0.05 * AU #inner radius of the disk
rout = 30 * AU #outer radius of the disk
sigma_in  = 2 * 10**5 #boundary condition at t=0
sigma_max = sigma_in*2 #boundary condition at t=final_time
sigma_min = 1 * 10**2
distance = rout - rin #distance from inner radius to outer radius

In [4]:
#Temporal discretization
max_years = 10
dyr = .1
dt = dyr * yrs2sec #timestep
final_time = max_years*yrs2sec + 1 #total diffusion time in seconds
t_save_interval = 10 #every ten years
t = np.arange(0, final_time, dt)

In [5]:
#Spacial discretization

n = 600 #number of steps / number of points to plot
dr = distance/n #distance between each spacial step
cont = True #Boolean for continuous alpha distribution

In [6]:
#Other Parameters

# St = .01 #Stokes Number
Q = 1
Tcor = 1800 #10% corundum
Tcalc = 1700 #10% calcium
Tfor = 1400 #80% forsterite

cor_ratio = .1
calc_ratio = .1
for_ratio = .8

phase_change = 1400

In [7]:
#Initialize grid spaces

dist = np.empty(n)
Omega = np.empty(n)
v_phi_dust = np.empty(n)
X = np.empty(n)

In [8]:
#Initialize Staggered Grid

dist_stag = np.empty(n-1)
Omega_stag = np.empty(n-1)

In [9]:
#Boundary Conditions

f_in = 0
f_out = 0

In [10]:
@numba.njit
def make_alpha_grid(cont):
    if(cont):
        alphas = np.linspace(alpha1, alpha3, n)
    else:
        alphas = np.empty(n)
        for i in range(n):
            if i < (n//3):
                alphas[i] = alpha1
            elif i < (2*n//3):
                alphas[i] = alpha2
            else:
                alphas[i] = alpha3
    return alphas

In [11]:
@numba.njit
def update_alpha(alphas, T):
    for idx, temp in enumerate(T):
        if(temp >= phase_change):
            alphas[idx] = alpha2 + 9e-3*min((temp-phase_change)/100, 1.)
        else:
            alphas[idx] = alpha2
    return alphas

In [12]:
def calc_dist_omega_stag():
    for i in range(1, n):
        dist_stag[i-1] = 0.5 * (dist[i] + dist[i-1])
        Omega_stag[i-1] = np.sqrt(G * Ms / dist_stag[i-1] ** 3)

In [13]:
def update_stag_grid(alphas, cs2):
    cs2_stag = np.array([0.5 * (cs2[i] + cs2[i-1]) for i in range(1, n)])
    alphas_stag = np.array([0.5 * (alphas[i] + alphas[i-1]) for i in range(1, n)])
    return (alphas_stag, cs2_stag)

In [14]:
@numba.njit
def update_nu(cs2, alphas):
    return alphas*cs2/Omega

In [15]:
@numba.njit
def update_St(cs2, cs2_stag, alphas, alpha_stag):
    dust = Q/(cs2_stag*alpha_stag)
    pebble = Q*10/(cs2*alphas)
    return (dust, pebble)

In [16]:
@numba.njit
def update_D(nu):
    return 12 * nu / (X ** 2)

In [17]:
def calc_dust_vel(St, cs2_stag, v_gas, press):
    press_stag = [0.5*(press[i]+press[i-1]) for i in range(1,n)]
    dPdr_stag = [(press[i] - press[i-1])/dist_stag[i-1] for i in range(1,n)]
    rho_stag = press_stag/cs2_stag
    v_dust = (St/(1+(St)**2)) * (1 / (rho_stag * Omega_stag))*dPdr_stag
    v_dust = np.append(v_dust, 0)
#     v_dust = v_dust + v_gas
    return v_dust

In [18]:
def calc_v_phi_gas(press,cs2):
    dPdr = np.zeros(n)
    for i in range(n):
        if (i == 0):
            dPdr[i] = (press[i+1] - press[i])/(dist[i])
        elif(i == n-1):
            dPdr[i] = (press[i] - press[i-1])/(dist[i])
        else:
            dPdr[i] = (press[i+1] - press[i-1])/(2*dist[i])
    rho = press/cs2
    v_phi_gas = np.sqrt(dist * ((G*Ms/(dist**2)) + (1/rho)*dPdr))
    return (rho, v_phi_gas)

In [19]:
def calc_init_params():
    """
    Calculate the initial parameters values for time t = 0.
    """
    sigma_gas = np.empty(n)
    sigma_dust = np.empty((3,n))
    sigma_pebble = np.zeros((3,n))
    sigma_evap_init = np.zeros((3,n))
    alphas = make_alpha_grid(cont)
    for i in range(n):
        dist[i] = (rin + (rout-rin)*i/(n-1))
        sigma_gas[i] = sigma_in * AU / dist[i]
        if (sigma_gas[i]>sigma_max):
            sigma_gas[i] = sigma_max
        if (dist[i]/AU > 15):
            sigma_gas[i] = sigma_min
        sigma_dust[0,i] = sigma_gas[i] * 0.005 * cor_ratio
        sigma_dust[1,i] = sigma_gas[i] * 0.005 * calc_ratio
        sigma_dust[2,i] = sigma_gas[i] * 0.005 * for_ratio
        X[i] = 2 * np.sqrt(dist[i])
        Omega[i] = np.sqrt(G * Ms / (dist[i] ** 3))
        v_phi_dust[i] = np.sqrt(Omega[i]**2 * dist[i])
    cs2, T, P, sigma_dust, sigma_evap, sigma_pebble = calc_thermal_struc(sigma_gas, sigma_dust, sigma_pebble, sigma_evap_init, alphas, Omega)
    calc_dist_omega_stag()
    alphas = update_alpha(alphas, T)
    alphas_stag, cs2_stag = update_stag_grid(alphas, cs2)
    St_dust = Q/(alphas_stag * cs2_stag)
    St_pebble = Q*10/(alphas*cs2)
    nu = alphas*cs2/Omega
    v_gas = calc_gas_vel(nu, sigma_gas, dist, dist_stag)
    v_dust = calc_dust_vel(St_dust, cs2_stag, v_gas, P)
    rho, v_phi_gas = calc_v_phi_gas(P, cs2)
    v_pebble = np.zeros(n)
    sigma_planet = np.zeros(n)
    D = 12 * nu / (X ** 2)
    f = (1.5 * X * sigma_gas)
    return (sigma_dust, sigma_gas, sigma_evap, sigma_pebble, sigma_planet, v_dust, v_gas, v_pebble, v_phi_gas, cs2, nu, D, f, alphas, alphas_stag, cs2_stag, St_dust, St_pebble, rho)

In [20]:
# @numba.njit
def calc_pebble_vel(v_dust):
#     Stoke = 0.05
#     press_stag = [0.5*(press[i]+press[i-1]) for i in range(1,n)]
#     dPdr_stag = [(press[i] - press[i-1])/dist_stag[i-1] for i in range(1,n)]
#     rho_stag = press_stag/cs2_stag
#     v_dust = (Stoke/(1+(Stoke)**2)) * (1 / (rho_stag * Omega_stag))*dPdr_stag
#     v_dust = np.append(v_dust, 0)
#     v_dust = v_dust + v_gas
    return v_dust * 10

In [21]:
@numba.njit
def calc_pebble_growth(sigma_pebble, sigma_dust):
    tau = 1.988e-7 * 1e4*yrs2sec / Omega
    sigma_pebble += sigma_dust * dt / tau
    sigma_dust -= sigma_dust * dt / tau
    return (sigma_pebble, sigma_dust)

In [22]:
# @numba.njit
def calc_planetessimal_growth(sigma_pebble, sigma_planet, St_pebble, v_phi_gas, rho):
    tau = 1.988e-7 * 1e4*yrs2sec / Omega
    v = v_phi_gas - v_phi_dust
    Rb = G*Ms/np.square(v)
    tb = Rb/v
    tau_f = St_pebble/Omega
    r_acc = np.sqrt(np.sqrt(4*tau_f/tb) * Rb)
    accretion_rate = np.pi * np.square(r_acc) * rho * (v + Omega*r_acc)
    sigma_planet += accretion_rate * dt / tau
    sigma_pebble[0,:] -= cor_ratio*sigma_planet * dt / tau
    sigma_pebble[1,:] -= calc_ratio*sigma_planet * dt / tau
    sigma_pebble[2,:] -= for_ratio*sigma_planet * dt / tau
    return (sigma_planet, sigma_pebble)

In [23]:
@numba.njit
def calc_gas_evol(f, dt, D):
    """Outputs the surface density at a specific dt."""
    df_dt = np.empty(n)
    for j in range(1, n-1):
        dX1 = X[j] - X[j-1]
        dX2 = X[j+1] - X[j]
        D1 = 0.5 * (D[j] + D[j-1])
        D2 = 0.5 * (D[j+1] + D[j])
        df_dt[j] = D1 * ((-(f[j] - f[j-1])/dX1**2)) + D2 * ((f[j+1]-f[j])/dX2**2)
    dX_final = X[-1]-X[-2]
    dX_in = X[1]-X[0]
    df_dt[0] = D[0] * (-(f[0] - f_in)/dX_in**2 + (f[1]-f[0])/dX_in**2)
    df_dt[n-1] = D[n-1] * (-(f[n-1] - f[n-2])/dX_final**2 + (f_out-f[n-1])/dX_final**2)
    f_new = f + df_dt * dt
    sigma_at_time = [2*f_new[k]/(3*X[k]) for k in range(n)]
    return (f_new, sigma_at_time)

In [24]:
# @numba.njit
def calc_disk_mass(sigma):
    """Calculate the mass at a specific dt"""
    disk_mass = 0
    for j in range(n):
        disk_mass += 2 * np.pi * dist[j] * dr * sigma_gas[j]
        disk_mass += 2 * np.pi * dist[j] * dr * sigma_dust[j]
#     print(f'The mass of the disk is {disk_mass}.')
    return disk_mass

In [25]:
def save2dir(**alpha_run):
    if(alpha_run):
        if(alpha_run == alpha1):
            output_dir = 'output_a1/'
            filename = 'output_a1/disk_'
        elif(alpha_run == alpha2):
            output_dir = 'output_a2/'
            filename = 'output_a2/disk_'
        else:
            output_dir = 'output_a3/'
            filename = 'output_a3/disk_'
    else:
        output_dir = 'output/'
        filename = 'output/disk_'
    return (output_dir, filename)

In [26]:
#Time Evolution Main

sigma_dust, sigma_gas, sigma_evap, sigma_pebble, sigma_planet, v_dust, v_gas, v_pebble, v_phi_gas, cs2, nu, D, f, alphas, alphas_stag, cs2_stag, St_dust, St_pebble, rho = calc_init_params()
output_dir, filename = save2dir()
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for i in range(len(t)):
    f_new, sigma_gas = calc_gas_evol(f, dt, D)
    sigma_pebble, sigma_dust = calc_pebble_growth(sigma_pebble, sigma_dust)
    sigma_planet, sigma_pebble = calc_planetessimal_growth(sigma_pebble, sigma_planet, St_pebble, v_phi_gas, rho)
    for el in range(3):
        sigma_dust[el,:] = calc_evol(sigma_gas, sigma_dust[el,:], nu, v_dust, dist, dt)
        sigma_pebble[el,:] = calc_evol(sigma_gas, sigma_pebble[el,:], nu, v_pebble, dist, dt)
        sigma_evap[el,:] = calc_evol(sigma_gas, sigma_evap[el,:], nu, v_gas, dist, dt)
    cs2, T, P, sigma_dust, sigma_evap, sigma_pebble = calc_thermal_struc(sigma_gas, sigma_dust, sigma_pebble, sigma_evap, alphas, Omega)
    alphas = update_alpha(alphas, T)
    alphas_stag, cs2_stag = update_stag_grid(alphas, cs2)
    nu = update_nu(cs2, alphas)
    D = update_D(nu)
    St_dust, St_pebble = update_St(cs2, cs2_stag, alphas, alphas_stag)
    v_dust = calc_dust_vel(St_dust, cs2_stag, v_gas, P)
    v_pebble = calc_pebble_vel(v_dust)
    v_gas = calc_gas_vel(nu, sigma_gas, dist, dist_stag)
    rho, v_phi_gas = calc_v_phi_gas(cs2,P)
    f = f_new
    if (i%(t_save_interval/dyr) == 0):
        output = np.transpose(np.vstack([dist/AU, sigma_gas, sigma_dust, sigma_pebble, sigma_evap, T, sigma_planet]))
        np.savetxt(filename + str(int(i*dyr)) + '.txt', output, delimiter=',', newline='\n')# save results to file

In [27]:
def plot():
    %matplotlib
    t_plot = [0]
    filename = 'output'
    for time in t_plot:
        df = pd.read_csv(filename + '/disk_' + str(int(t_plot[-1])) + '.txt', delimiter=',', header=None)
        sigma_gas_plot = df[1].to_numpy().astype(np.float)
        cor_dust = df[2].to_numpy().astype(np.float)
        calc_dust = df[3].to_numpy().astype(np.float)
        for_dust = df[4].to_numpy().astype(np.float)
        cor_peb = df[8].to_numpy().astype(np.float)
        calc_peb = df[9].to_numpy().astype(np.float)
        for_peb = df[10].to_numpy().astype(np.float)
        temp_plot = df[12].to_numpy().astype(np.float)
        press_plot = df[13].to_numpy().astype(np.float)
        sigma_dust_plot = cor_dust+calc_dust+for_dust
        sigma_peb_plot = cor_peb+calc_peb+for_peb
        dg_ratio = sigma_dust_plot/sigma_gas_plot
        fig, axes = plt.subplots(2,3, figsize=(12,7.5))
        ax1 = axes[0][0]
        ax2 = axes[1][0]
        ax3 = axes[0][1]
        ax4 = axes[1][1]
        ax5 = axes[1][2]
        ax6 = axes[0][2]
        for row in axes:
            for col in row:
                col.set_xlim([0,10])
                col.tick_params(direction='in', length=3,   which='major', top=True, right=True)
                col.tick_params(direction='in', length=1.5, which='minor', top=True, right=True)
                col.set_xlabel('Distance (AU)')
        ax1.set_ylabel('Gas (kg/m^3)')
        ax1.set_yscale('log')
        ax1.set_ylim(1e4,1e6)
        ax1.plot(dist/AU, sigma_gas_plot)
        ax2.set_ylabel('Dust')
        ax2.set_yscale('log')
#         ax2.set_ylim(1e-7,1e3)
        ax2.plot(dist/AU, cor_dust, label='Aluminum', color='green')
        ax2.plot(dist/AU, calc_dust, label='Calcium', color='red')
        ax2.plot(dist/AU, for_dust, label='Magnesium', color='blue')
        ax2.legend(fontsize='x-small')
        ax3.set_title(f'{int(time)} years with {filename[14:]}')
        ax3.set_ylabel('Dust/Gas')
        ax3.set_yscale('log')
        ax3.plot(dist/AU, dg_ratio)
        ax4.set_ylabel('Pebbles')
        ax4.set_yscale('log')
        ax4.set_ylim(1e-4,1e-1)
        ax4.plot(dist/AU, cor_peb, label='Aluminum', color='green')
        ax4.plot(dist/AU, calc_peb, label='Calcium', color='red')
        ax4.plot(dist/AU, for_peb, label='Magnesium', color='blue')
        ax4.legend(fontsize='x-small')
        ax5.set_ylabel('Pressure')
        ax5.set_ylim(1e-2,1e4)
        ax5.set_yscale('log')
        ax5.plot(dist/AU, press_plot, label='gas', color='purple')
        ax6.set_ylabel('Temperature (K)')
        ax6.set_yscale('log')
        ax6.set_ylim(1e1,1e4)
        ax6.plot(dist/AU, temp_plot)
    plt.show()
    plt.tight_layout()
    plt.savefig(f'./Evol_{int(t_plot[-1])}_{filename[14:]}.pdf', transparent=True)

In [32]:
def plot_planet():
    %matplotlib
    t_plot = [0, 10]
    filename = 'output'
    for time in t_plot:
        df = pd.read_csv(filename + '/disk_' + str(int(t_plot[-1])) + '.txt', delimiter=',', header=None)
        planet_plot = df[12].to_numpy().astype(np.float)
        plt.title('Planetessimal Evolution')
        plt.yscale('log')
        plt.xlabel('Distance (AU)')
        plt.ylabel('Surface Density')
        plt.plot(dist/AU, planet_plot)

In [33]:
plot_planet()

Using matplotlib backend: MacOSX


In [ ]:
#         cm = pylab.get_cmap('jet') #gist_rainbow
#         plt.rcParams['font.family']='sans-serif'
#         plt.rcParams['axes.linewidth']= 1.0
#         plt.rcParams['font.size']     = 14
#         plt.rcParams['figure.figsize']= 4*1.414, 4
#         rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
#         rc('text',usetex=False)
#         rc('text.latex', preamble=r'\usepackage{sfmath}')